In [2]:
%conda install -c conda-forge beautifulsoup4


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         518 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0

The following packages will be UPDATED:

  ca-certificate

In [50]:
%conda install -c conda-forge geopy

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [7]:
import urllib.request
from  geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "html.parser")
#print(soup.prettify())
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
#right_table
data=[]
dataNeighbours={}

for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) > 1:
        dataNeighbours[cells[0].text] = [cells[1].text]
#print (dataNeighbours['C01\n'])
for key in dataNeighbours:
    #print(dataNeighbours[key])
    for val in dataNeighbours[key]:
        valArr = val.split(',')
        for rr in valArr:
            data.append([key.replace('\n',''), rr])
dfObj = pd.DataFrame(data, columns=['DistNum', 'Neighbourhoods'])  
dfObj['CityName'] = 'Toronto'
dfObj['latitude'] = 'Null'
dfObj['longitude'] = 'Null'
for index, row in dfObj.iterrows():
    #print(row['Neighbourhoods'], row['CityName'])
    address = row['Neighbourhoods']+', '+row['CityName']+' Canada'
    geolocator = Nominatim(user_agent="testApp")
    location = geolocator.geocode(address)
    if location is not None:
        dfObj.at[index, 'latitude'] = location.latitude
        dfObj.at[index, 'longitude'] = location.longitude
dfObj

,DistNum,Neighbourhoods,CityName,latitude,longitude
0,C01,Downtown,Toronto,43.6563,-79.3809
1,C01,Harbourfront,Toronto,43.6401,-79.3801
2,C01,Little Italy,Toronto,43.6552,-79.4149
3,C01,Little Portugal,Toronto,43.6474,-79.4311
4,C01,Dufferin Grove,Toronto,43.6536,-79.4264
...,...,...,...,...,...
220,W10,Thistletown - Beaumond Heights,Toronto,Null,Null
221,W10,Smithfield: Mount Olive-Silverstone-Jamestown,Toronto,Null,Null
222,W10,The Elms (Elms-Old Rexdale),Toronto,43.7108,-79.5654
223,W10,West Humber - Clairville,Toronto,Null,Null


In [9]:
indexNames = dfObj[dfObj['latitude'] == 'Null'].index
dfObj.drop(indexNames , inplace=True)
dfObj

,DistNum,Neighbourhoods,CityName,latitude,longitude
0,C01,Downtown,Toronto,43.6563,-79.3809
1,C01,Harbourfront,Toronto,43.6401,-79.3801
2,C01,Little Italy,Toronto,43.6552,-79.4149
3,C01,Little Portugal,Toronto,43.6474,-79.4311
4,C01,Dufferin Grove,Toronto,43.6536,-79.4264
...,...,...,...,...,...
216,W09,Kingsview Village,Toronto,43.6995,-79.5563
217,W09,Westmount\n,Toronto,43.6936,-79.521
218,W10,Rexdale,Toronto,43.7214,-79.5655
222,W10,The Elms (Elms-Old Rexdale),Toronto,43.7108,-79.5654


In [19]:
import folium
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, neighbour, city in zip(dfObj['latitude'], dfObj['longitude'], dfObj['Neighbourhoods'], dfObj['CityName']):
    label = '{}, {}'.format(neighbour, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto